In [1]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

In [2]:
es_client.info()

ObjectApiResponse({'name': 'b35a12ea461e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'e3owGagcTGifEHcVgbq4OA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [3]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
from tqdm.auto import tqdm

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:02<00:00, 360.48it/s]


In [10]:
query="How do I execute a command in a running docker container?"
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)
    


In [15]:
response['hits']['hits'][0]['_score']

84.050095

In [16]:
query="How do I execute a command in a running docker container?"
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
            }
        }
    }}

response = es_client.search(index=index_name, body=search_query)

In [18]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'zqaNXJABnhyN6z5b3uGZ',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': '7aaNXJABnhyN6z5b3uHk',
  '_score': 51.04628,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_

In [31]:

context=""
for resp in response['hits']['hits']:
    question=resp['_source']['question']
    text=resp['_source']['text']
    context=context+f"""
Q: {question}
A: {text}
""".strip() +"\n\n"

In [32]:
print(context.strip())

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [33]:
question="How do I execute a command in a running docker container?"
prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [34]:
len(prompt_template)

1462

In [35]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [39]:
len(encoding.encode(prompt_template))

322

In [41]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv("../../.env")
client=OpenAI()
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-3.5-turbo-0125',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [42]:
response=llm(prompt_template)

In [43]:
response

'To execute a command in a running docker container, you can use the `docker exec` command. First, use `docker ps` to find the container ID of the specific container. Then, you can use the following syntax:\ndocker exec -it <container-id> bash'